In [1]:
# Here, we use the developed tool to generate images of the heatmaps associated with the paper's results section

In [1]:
import importlib
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
import random
from operator import add
from graphviz import Source, render
import pydot

import datetime
import time

import sys
sys.path.append('py_modules') # below modules are in this folder
#print(sys.path)
import setup_nx # your own module, setup.nx.py
importlib.reload(setup_nx)
import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm

In [2]:
# define function to display imgs generated
from IPython.display import display
from PIL import Image

# shorten png printing commands into one function call
def printpng(filename):
    im = Image.open(filename) 
    #print(im.size) # (pixel width,pixel height)
    width=im.size[0]
    height=im.size[1]
    im.draft('RGB',(width/1,height/1)) 
    px = im.load()
    display(im)
    # reference on img.draft func: https://stackoverflow.com/questions/57663734/how-to-speed-up-image-loading-in-pillow-python
    
    return

In [3]:
# [Essential] specify input feeder data for IEEE 123-node test feeder

# Enter the path/name of the impedance model data (excel file)
filepath = "feeder_impedance_models/"
modelpath = filepath + "004_GB_IEEE123_OPAL.xls"

# load data is not actually used in assessment, but the feeder object builds variables to hold load info
loadfolder = filepath
loadpath = filepath + '004_123NF_PVpen100_nocloud_minutewise_whead.csv'
headerpath = filepath + '004_GB_IEEE123_time_header.csv'
load_data = loadpath

#file_name = string specifying name of dot file created when make_graph() is called
file_name = '123NF'
#file_name = 'demo123NF.dot'

# Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4160
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_150'
timesteps = 1

# initialize some variables
ts = time.time()
print(datetime.datetime.fromtimestamp(ts))
plot = 0 #turn plot on/off
depths = {}
leaves = []

2020-11-23 10:47:41.929618


In [4]:
# [ESSENTIAL] create feeder object

fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
R,X=hm.createRXmatrices_3ph(fin_feeder, node_index_map,depths)

print('depths=',depths) # check this is populated, lists how far each node is from substation
print('depths length=',len(depths))

# print list of first 10 buses in network
count = 0 
for i in fin_feeder.network:    
    print(i) 
    count += 1
    if count >= 10:
        break
    
import csv  # jaimie do you need this?
graph = fin_feeder.network 


  File "C:\Users\jaimi\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\jaimi\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\jaimi\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\jaimi\Anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\jaimi\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
    self.io_loop.start()
  File "C:\Users\jaimi\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 153, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\jaimi\Anaconda3\lib\asyncio\base_events.py", line 538, in run_forever
    self._run_once()
  File "C:\Users\jaimi\Anaconda3\lib\asyncio\base_events.py", line 1782, in _run_once
    handle._run()
  File "C:\Users\jaimi\Anaconda3\lib\asyncio\events.py", line 88, in _r

Finished initializing feeder
depths= {'bus_150': 0, 'bus_149': 1, 'bus_1': 2, 'bus_2': 3, 'bus_3': 3, 'bus_7': 3, 'bus_4': 4, 'bus_5': 4, 'bus_8': 4, 'bus_6': 5, 'bus_12': 5, 'bus_14': 5, 'bus_13': 5, 'bus_9': 6, 'bus_11': 6, 'bus_10': 6, 'bus_34': 6, 'bus_18': 6, 'bus_152': 6, 'bus_15': 7, 'bus_19': 7, 'bus_21': 7, 'bus_135': 7, 'bus_52': 7, 'bus_16': 8, 'bus_17': 8, 'bus_20': 8, 'bus_22': 8, 'bus_23': 8, 'bus_35': 8, 'bus_53': 8, 'bus_24': 9, 'bus_25': 9, 'bus_36': 9, 'bus_40': 9, 'bus_54': 9, 'bus_26': 10, 'bus_28': 10, 'bus_37': 10, 'bus_38': 10, 'bus_41': 10, 'bus_42': 10, 'bus_55': 10, 'bus_57': 10, 'bus_27': 11, 'bus_31': 11, 'bus_29': 11, 'bus_39': 11, 'bus_43': 11, 'bus_44': 11, 'bus_56': 11, 'bus_58': 11, 'bus_60': 11, 'bus_33': 12, 'bus_32': 12, 'bus_30': 12, 'bus_45': 12, 'bus_47': 12, 'bus_59': 12, 'bus_61': 12, 'bus_62': 12, 'bus_160': 12, 'bus_250': 13, 'bus_46': 13, 'bus_48': 13, 'bus_49': 13, 'bus_610': 13, 'bus_610_': 13, 'bus_63': 13, 'bus_67': 13, 'bus_251': 14, 'bu

In [6]:
#printpng('demo_generated_figs/123NF_blank.png')

In [7]:
# Motivating example configuration 2 (same configuration as CPP process except for node 60)
all_act_locs=['bus_60','bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']
# ---------- mark actuator config -------------
vis.markActuatorConfig(all_act_locs,fin_feeder, file_name)
(graph,) = pydot.graph_from_dot_file('generated_figs/'+'actConfig_'+ file_name) # requires 'import pydot'
graph.write_png('generated_figs/'+'config_60.png')
#printpng('demo_generated_figs/config_60.png')

# Motivating example configuration 1 (same configuration as CPP process except for node 60)
all_act_locs=['bus_61','bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']
# ---------- mark actuator config -------------
vis.markActuatorConfig(all_act_locs,fin_feeder, file_name)
(graph,) = pydot.graph_from_dot_file('generated_figs/'+'actConfig_'+ file_name) # requires 'import pydot'
graph.write_png('generated_figs/'+'config_61.png')
#printpng('demo_gehttp://localhost:8890/notebooks/Documents/MATLAB/Research/energise/local_heatMap_pythonCode3/driver_code_forPaper.ipynb#nerated_figs/config_61.png')

In [8]:
# Neighborhood scenario step 7 heatmap, img is of whole feeder

# heatmap/collect stable configs for 7th step only:
set_act_locs = ['bus_82','bus_87','bus_76','bus_49','bus_41','bus_46']
set_perf = ['bus_77','bus_77','bus_77','bus_44','bus_44','bus_44']
addon_locs = ['bus_60']
addon_perf_nodes = ['bus_66']

t = time.time()
# set_act_locs and set_perf are existing pairs, addon_locs and addon_perf_nodes are pairs to add after produce heatmap
lst_feas_configs, lzn_error_run_sum, heatMapNames = hm.runHeatMapProcess(fin_feeder, set_act_locs, set_perf, addon_locs, addon_perf_nodes, node_index_map, substation_name, depths, file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
# lst_feas_configs is list of dictionaries
elapsed = time.time() - t
print('Time elapsed=',elapsed)
# takes about an hour to run

(graph,) = pydot.graph_from_dot_file('generated_figs/'+'NPP_heatmap_step0_' + file_name) # requires 'import pydot'
graph.write_png('generated_figs/'+'RHP_nbhd_wideblue.png')
#printpng('demo_generated_figs/RHP_nbhd_wideblue.png')

evaluating actuator node at  ['bus_1', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44', 'bus_44']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator node at  ['bus_10', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44', 'bus_44']
Config good!
num feas= [[177.]]
num tried= 225
evaluating actuator node at  ['bus_100', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44', 'bus_44']
Config good!
num feas= [[63.]]
num tried= 225
evaluating actuator node at  ['bus_101', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44', 'bus_44']
Config good!
num feas= [[64.]]
num tried= 225
evaluating 

No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator node at  ['bus_22', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44', 'bus_44']
Config good!
num feas= [[182.]]
num tried= 225
evaluating actuator node at  ['bus_23', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44', 'bus_44']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator node at  ['bus_24', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44', 'bus_44']
Config good!
num feas= [[175.]]
num tried= 225
evaluating actuator node at  ['bus_25', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44'

No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator node at  ['bus_5', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44', 'bus_44']
Config good!
num feas= [[179.]]
num tried= 225
evaluating actuator node at  ['bus_50', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44', 'bus_44']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator node at  ['bus_51', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44', 'bus_44']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator node at  ['bus_52', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77

No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator node at  ['bus_80', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44', 'bus_44']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator node at  ['bus_81', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44', 'bus_44']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator node at  ['bus_83', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus_77', 'bus_77', 'bus_77', 'bus_44', 'bus_44', 'bus_44']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator node at  ['bus_84', 'bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'] ,
 performance node at  ['bus_66', 'bus

In [9]:
# Neighborhood scenario step 7 heatmap, image is after cropped and neighborhoods 1 and 2 marked
#printpng('demo_generated_figs/results_figs/github_figs/RHP_nbhd_wideblue_cropped_markup.png')

In [5]:
# Neighborhood scenario branch analysis
# Code to produce data for Table I

# The NPP on the neighborhood configuration for steps 1 through 7 takes a long time to run (evaluating about 800 configurations), so instead
# load data 'lst_feas_configs' from .pkl file
import pickle
filename='nbhs_RHP_lstfeasconfigs.pkl'
with open(filename,'rb') as f:  # Python 3: open(..., 'rb')
    lst_feas_configs = pickle.load(f)
    
# convert lst_feas_configs (list of dictionaries) to newlst_feas_configs (a list of lists)
# input a list of feasible actuator configurations where each configuration is its own list (ie a list of lists)"
length=len(lst_feas_configs)
newlst_feas_configs = []
numact_lst=[] # store number of actuators in separate list
i=0
for dic in lst_feas_configs:
    #print('dic=',dic)
    #print(type(dic))
    i+=1
    if i<length-1:
        #print('dic[act]=',str(dic['act']))
        newlst_feas_configs += [dic['act']]
        numact_lst+=[dic['numfeas']]
        
# create 'placed' variable
# placed is an ordered list of lists, where the k'th ele of the list is the actuators placed at that step
set_act_locs = []
set_perf = []
addon_locs = ['bus_82','bus_87','bus_76','bus_49','bus_41','bus_46','bus_66']
addon_perf_nodes = ['bus_77','bus_77','bus_77','bus_44','bus_44','bus_44','bus_66']

acts=set_act_locs + addon_locs
placed=[]
k=0
while k<=len(acts):
    placed+=[acts[0:k]]
    k+=1
print(placed)

# ----- run assign_network_branches ---------"
branches = vis.assign_network_branches(fin_feeder, substation_name)
vis.mark_network_branches(fin_feeder, branches, file_name, substation_name,depths)
Source.from_file('branch_key:' + file_name)
print('branches table:') # format into a table with index and branch nodes
i=0
for br in branches:
    print(i,',',br)
    i+=1

# ----- run find_good_branches ---------"
# the 'best' branch is the branch with the most actuators from feas configs on it across all configurations
num_good_branches = 8 # return the n best branches
vis.find_good_branches(newlst_feas_configs, branches, num_good_branches, placed)


[[], ['bus_82'], ['bus_82', 'bus_87'], ['bus_82', 'bus_87', 'bus_76'], ['bus_82', 'bus_87', 'bus_76', 'bus_49'], ['bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41'], ['bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46'], ['bus_82', 'bus_87', 'bus_76', 'bus_49', 'bus_41', 'bus_46', 'bus_66']]
branches table:
0 , ['bus_2']
1 , ['bus_4']
2 , ['bus_3', 'bus_5', 'bus_6']
3 , ['bus_12']
4 , ['bus_19', 'bus_20']
5 , ['bus_37']
6 , ['bus_36', 'bus_38', 'bus_39']
7 , ['bus_41']
8 , ['bus_43']
9 , ['bus_45', 'bus_46']
10 , ['bus_48']
11 , ['bus_135', 'bus_35', 'bus_40', 'bus_42', 'bus_44', 'bus_47', 'bus_49', 'bus_50', 'bus_51', 'bus_151']
12 , ['bus_22']
13 , ['bus_24']
14 , ['bus_27', 'bus_33']
15 , ['bus_26', 'bus_31', 'bus_32']
16 , ['bus_18', 'bus_21', 'bus_23', 'bus_25', 'bus_28', 'bus_29', 'bus_30', 'bus_250', 'bus_251']
17 , ['bus_55', 'bus_56']
18 , ['bus_58', 'bus_59']
19 , ['bus_610']
20 , ['bus_61', 'bus_610_']
21 , ['bus_68', 'bus_69', 'bus_70', 'bus_71']
22 , ['bus_73', 'bus_

In [6]:
# non-auto CPP Process heatmap, Motivating example

# these vars arent used
set_act_locs = ['bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']
set_perf = ['bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']
addon_locs = ['bus_60']
addon_perf_nodes = ['bus_60']

# ------------ run placeMaxColocActs_stopAtInfeas -----------
# place colocated actuators until an infeasible loc is tested, then call find_good_colocated and return 
# produces colormap associated with "motex" motivating example
t = time.time()
max_act_config_stopInfeas=hm.placeMaxColocActs_stopAtInfeas(fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
elapsed = time.time() - t
print('Time elapsed=',elapsed)
# takes about 5 hours

#printpng('demo_generated_figs/heatmap_colocated_123NF.png')


evaluating actuator and performance node colocated at  ['bus_197']
Config good!
num feas= [[124.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_55', 'bus_197']
Config good!
num feas= [[137.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_5', 'bus_197', 'bus_55']
Config good!
num feas= [[143.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_113', 'bus_197', 'bus_55', 'bus_5']
Config good!
num feas= [[169.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_33', 'bus_197', 'bus_55', 'bus_5', 'bus_113']
Config good!
num feas= [[156.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_96', 'bus_197', 'bus_55', 'bus_5', 'bus_113', 'bus_33']
Config good!
num feas= [[132.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_60', 'bus_197', 'bus_55', 'bus_5', 'bus_113', 'bus_33', 'bus_96']
No F found for config --> bad config
num

Config good!
num feas= [[28.]]
num tried= 225
evaluating act and perf colocated at  ['bus_58']
Config good!
num feas= [[132.]]
num tried= 225
evaluating act and perf colocated at  ['bus_59']
Config good!
num feas= [[132.]]
num tried= 225
evaluating act and perf colocated at  ['bus_6']
Config good!
num feas= [[132.]]
num tried= 225
evaluating act and perf colocated at  ['bus_60']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating act and perf colocated at  ['bus_61']
Config good!
num feas= [[63.]]
num tried= 225
evaluating act and perf colocated at  ['bus_610']
Config good!
num feas= [[63.]]
num tried= 225
evaluating act and perf colocated at  ['bus_62']
Config good!
num feas= [[101.]]
num tried= 225
evaluating act and perf colocated at  ['bus_63']
Config good!
num feas= [[110.]]
num tried= 225
evaluating act and perf colocated at  ['bus_64']
Config good!
num feas= [[120.]]
num tried= 225
evaluating act and perf colocated at  ['bus_65']
Config good!
num feas

In [5]:
# Code to produce data for Table II and III
# Auto-CPP, for random seeds 3,4,5,6,7,8
# ------ Place place_max_colocated_acts -------------
t = time.time()
seed=2
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
seed=3
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
seed=4
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
seed=5
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
seed=6
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
seed=7
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
seed=8
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

elapsed = time.time() - t
print('Time elapsed=',elapsed)

# printpng('demo_generated_figs/actConfig_max_coloc_seed2.png')
# printpng('demo_generated_figs/actConfig_max_coloc_seed3.png')
# printpng('demo_generated_figs/actConfig_max_coloc_seed4.png')
# printpng('demo_generated_figs/actConfig_max_coloc_seed5.png')
# printpng('demo_generated_figs/actConfig_max_coloc_seed6.png')
# printpng('demo_generated_figs/actConfig_max_coloc_seed8.png')


----------- Seed=2 ----------------------------
evaluating actuator and performance node colocated at  ['bus_96']
Config good!
num feas= [[210.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_85', 'bus_96']
Config good!
num feas= [[196.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_97', 'bus_96', 'bus_85']
Config good!
num feas= [[196.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_83', 'bus_96', 'bus_85', 'bus_97']
Config good!
num feas= [[156.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_105', 'bus_96', 'bus_85', 'bus_97', 'bus_83']
Config good!
num feas= [[13.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_11', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105']
Config good!
num feas= [[11.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_109', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_1

No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_37', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_152', 'bus_197', 'bus_6', 'bus_102', 'bus_58', 'bus_4']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_44', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_152', 'bus_197', 'bus_6', 'bus_102', 'bus_58', 'bus_4']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_56', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_152', 'bus_197', 'bus_6', 'bus_102', 'bus_58', 'bus_4']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_16', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_152', 'bus_197', 'bus_6', 'b

No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_46', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_152', 'bus_197', 'bus_6', 'bus_102', 'bus_58', 'bus_4', 'bus_2']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_99', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_152', 'bus_197', 'bus_6', 'bus_102', 'bus_58', 'bus_4', 'bus_2']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_60', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_152', 'bus_197', 'bus_6', 'bus_102', 'bus_58', 'bus_4', 'bus_2']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_88', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_1

No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_57', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_152', 'bus_197', 'bus_6', 'bus_102', 'bus_58', 'bus_4', 'bus_2']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_113', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_152', 'bus_197', 'bus_6', 'bus_102', 'bus_58', 'bus_4', 'bus_2']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_3', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_152', 'bus_197', 'bus_6', 'bus_102', 'bus_58', 'bus_4', 'bus_2']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_32', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_1

No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_101', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_152', 'bus_197', 'bus_6', 'bus_102', 'bus_58', 'bus_4', 'bus_2']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_100', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_152', 'bus_197', 'bus_6', 'bus_102', 'bus_58', 'bus_4', 'bus_2']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_27', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus_152', 'bus_197', 'bus_6', 'bus_102', 'bus_58', 'bus_4', 'bus_2']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_29', 'bus_96', 'bus_85', 'bus_97', 'bus_83', 'bus_105', 'bus_11', 'bus

...

No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_49', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109', 'bus_135', 'bus_25', 'bus_88', 'bus_20', 'bus_5', 'bus_9', 'bus_18', 'bus_66', 'bus_107', 'bus_102', 'bus_55', 'bus_2', 'bus_26', 'bus_79', 'bus_82', 'bus_6']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_70', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109', 'bus_135', 'bus_25', 'bus_88', 'bus_20', 'bus_5', 'bus_9', 'bus_18', 'bus_66', 'bus_107', 'bus_102', 'bus_55', 'bus_2', 'bus_26', 'bus_79', 'bus_82', 'bus_6']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_56', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109'

No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_112', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109', 'bus_135', 'bus_25', 'bus_88', 'bus_20', 'bus_5', 'bus_9', 'bus_18', 'bus_66', 'bus_107', 'bus_102', 'bus_55', 'bus_2', 'bus_26', 'bus_79', 'bus_82', 'bus_6']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_33', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109', 'bus_135', 'bus_25', 'bus_88', 'bus_20', 'bus_5', 'bus_9', 'bus_18', 'bus_66', 'bus_107', 'bus_102', 'bus_55', 'bus_2', 'bus_26', 'bus_79', 'bus_82', 'bus_6']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_100', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_10

No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_91', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109', 'bus_135', 'bus_25', 'bus_88', 'bus_20', 'bus_5', 'bus_9', 'bus_18', 'bus_66', 'bus_107', 'bus_102', 'bus_55', 'bus_2', 'bus_26', 'bus_79', 'bus_82', 'bus_6']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_61', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109', 'bus_135', 'bus_25', 'bus_88', 'bus_20', 'bus_5', 'bus_9', 'bus_18', 'bus_66', 'bus_107', 'bus_102', 'bus_55', 'bus_2', 'bus_26', 'bus_79', 'bus_82', 'bus_6']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_22', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109'

No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_94', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109', 'bus_135', 'bus_25', 'bus_88', 'bus_20', 'bus_5', 'bus_9', 'bus_18', 'bus_66', 'bus_107', 'bus_102', 'bus_55', 'bus_2', 'bus_26', 'bus_79', 'bus_82', 'bus_6']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_15', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109', 'bus_135', 'bus_25', 'bus_88', 'bus_20', 'bus_5', 'bus_9', 'bus_18', 'bus_66', 'bus_107', 'bus_102', 'bus_55', 'bus_2', 'bus_26', 'bus_79', 'bus_82', 'bus_6']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_89', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109'

No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_36', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109', 'bus_135', 'bus_25', 'bus_88', 'bus_20', 'bus_5', 'bus_9', 'bus_18', 'bus_66', 'bus_107', 'bus_102', 'bus_55', 'bus_2', 'bus_26', 'bus_79', 'bus_82', 'bus_6']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_84', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109', 'bus_135', 'bus_25', 'bus_88', 'bus_20', 'bus_5', 'bus_9', 'bus_18', 'bus_66', 'bus_107', 'bus_102', 'bus_55', 'bus_2', 'bus_26', 'bus_79', 'bus_82', 'bus_6']
No F found for config --> bad config
num feas= [[0.]]
num tried= 225
evaluating actuator and performance node colocated at  ['bus_75', 'bus_19', 'bus_32', 'bus_99', 'bus_34', 'bus_113', 'bus_16', 'bus_71', 'bus_103', 'bus_109'